In [3]:
import pickle
import torch as t
from torch.utils import data
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
with open('./ranked list truncation/data_prep/my_results/BM25_results/split_1/BM25_train_s1.pkl', 'rb') as f: train_data_raw = pickle.load(f)

with open('./ranked list truncation/data_prep/my_results/BM25_results/split_1/BM25_test_s1.pkl', 'rb') as f: test_data_raw = pickle.load(f)

with open('./ranked list truncation/data_prep/robust04_data/robust04_gt.pkl', 'rb') as f: gt = pickle.load(f)

In [7]:
train_data_raw['301']['retrieved_documents'][0]

{'doc_id': 'FBIS3-21961',
 'rank': 1,
 'bm25_score': 5.67759784,
 'norm_bm25_score': 3.2061266261914216,
 'is_relevant': True}

In [26]:
X_train, y_train = [], []
for key in train_data_raw:
    scores = [train_data_raw[key]['retrieved_documents'][i]['norm_bm25_score'] for i in range(300)]
    is_rel = [1 if train_data_raw[key]['retrieved_documents'][i]['is_relevant'] else 0 for i in range(300)]
    X_train.append(scores)
    y_train.append(is_rel)

In [27]:
X_train = t.unsqueeze(t.Tensor(X_train), dim=1)
y_train = t.Tensor(y_train)

In [31]:
with open('./ranked list truncation/data_prep/my_results/drmm_tks_results/split_1/drmm_tks_train_s1.pkl', 'rb') as f: train_data_raw = pickle.load(f)

In [32]:
train_data_raw['301'][0]

{'doc_id': 'FBIS3-41131', 'score': 6.436135292053223}

In [6]:
a = t.Tensor([[1], [2], [3]])
b = t.sum(a[:2])
b / 2

tensor(1.5000)

In [11]:
if t.tensor(0) == 0: print('1')

1


In [12]:
f = lambda x, *y: x ** 2
a.map_(a, f)
a

tensor([1., 4., 4.])

In [17]:
a = t.tensor([1,2,3])

In [1]:
import torch as t
a = t.randn(2, 3, 1)
a

tensor([[[ 1.1999e+00],
         [-8.7357e-01],
         [-2.8038e-02]],

        [[ 1.5940e+00],
         [-3.9927e-04],
         [-1.4029e+00]]])

In [2]:
b = t.randn(2, 3, 4)
a = t.cat((a, b), dim=2)

In [3]:
a

tensor([[[ 1.1999e+00,  7.5063e-01, -4.7991e-01, -2.8267e-01,  1.0497e+00],
         [-8.7357e-01, -1.5994e+00,  1.1305e+00, -9.9166e-01,  1.5842e+00],
         [-2.8038e-02, -5.1788e-01, -5.9645e-01,  1.8467e+00,  9.8149e-01]],

        [[ 1.5940e+00, -4.8038e-01, -1.3766e+00, -3.9622e-01,  7.0838e-01],
         [-3.9927e-04,  1.0944e-01,  1.7177e+00, -1.2022e+00,  3.1233e-01],
         [-1.4029e+00, -7.2118e-01,  1.5183e+00,  2.2340e-01,  7.8001e-01]]])

##  bicut数据分query存储

In [1]:
import pickle
import numpy as np

In [2]:
import os

In [ ]:
BM25_BASE = '/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/my_results/BM25_results'
DRMM_BASE = '/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/my_results/drmm_results'
DRMM_TKS_BASE = '/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/my_results/drmm_tks_results'
STATS_BASE = '/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/statics'
GT_PATH = '/home/LAB/wangd/graduation_project/ranked list truncation/data_prep/robust04_data/robust04_gt.pkl'

def data_prepare(dataset_name: str, split: int):
    if dataset_name == 'BM25':
        with open('{}/split_{}/BM25_train_s{}.pkl'.format(BM25_BASE, split, split), 'rb') as f:
            train_data_raw = pickle.load(f)
        with open('{}/split_{}/BM25_test_s{}.pkl'.format(BM25_BASE, split, split), 'rb') as f:
            test_data_raw = pickle.load(f)
        with open('{}/bicut_bm25_input.pkl'.format(STATS_BASE), 'rb') as f:
            stats_bm25 = pickle.load(f)

        X_train, X_test, y_train, y_test = [], [], [], []
        for key in train_data_raw:
            scores = np.array([train_data_raw[key]['retrieved_documents'][i]['norm_bm25_score']
                        for i in range(300)])
            stats = np.array(stats_bm25[key])
            input_features = np.column_stack((scores, stats))
            is_rel = [1 if train_data_raw[key]['retrieved_documents'][i]['is_relevant'] else 0
                        for i in range(300)]
            if not os.path.exits('{}/bicut_bm25_s{}_train/'.format(STATS_BASE, split)): 
                os.mkdir('{}/bicut_bm25_s{}_train/'.format(STATS_BASE, split))
            with open('{}/bicut_bm25_s{}_train/qid_{}.pkl'.format(STATS_BASE, split, key), 'wb') as f:
                pickle.dump((input_features, is_rel), f)

        for key in test_data_raw:
            scores = np.array([test_data_raw[key]['retrieved_documents'][i]['norm_bm25_score']
                        for i in range(300)])
            stats = np.array(stats_bm25[key])
            input_features = np.column_stack((scores, stats))
            is_rel = [1 if test_data_raw[key]['retrieved_documents'][i]['is_relevant'] else 0
                        for i in range(300)]
            if not os.path.exits('{}/bicut_bm25_s{}_test/'.format(STATS_BASE, split)): 
                os.mkdir('{}/bicut_bm25_s{}_test/'.format(STATS_BASE, split))
            with open('{}/bicut_bm25_s{}_test/qid_{}.pkl'.format(STATS_BASE, split, key), 'wb') as f:
                pickle.dump((input_features, is_rel), f)

    elif dataset_name == 'DRMM':
        with open('{}/split_{}/drmm_train_s{}.pkl'.format(DRMM_BASE, split, split), 'rb') as f:
            train_data_raw = pickle.load(f)
        with open('{}/split_{}/drmm_test_s{}.pkl'.format(DRMM_BASE, split, split), 'rb') as f:
            test_data_raw = pickle.load(f)
        with open('{}/bicut_drmm_input.pkl'.format(STATS_BASE), 'rb') as f:
            stats_drmm = pickle.load(f)
        with open(GT_PATH, 'rb') as f:
            gt = pickle.load(f)
            for key in gt:
                gt[key] = set(gt[key])

        X_train, X_test, y_train, y_test = [], [], [], []
        for key in train_data_raw:
            scores = np.array([train_data_raw[key][i]['score'] for i in range(300)])
            stats = np.array(stats_drmm[key])
            input_features = np.column_stack((scores, stats))
            is_rel = [1 if train_data_raw[key][i]['doc_id'] in gt[key] else 0
                        for i in range(300)]
            if not os.path.exits('{}/bicut_drmm_s{}_train/'.format(STATS_BASE, split)): 
                os.mkdir('{}/bicut_drmm_s{}_train/'.format(STATS_BASE, split))
            with open('{}/bicut_drmm_s{}_train/qid_{}.pkl'.format(STATS_BASE, split, key), 'wb') as f:
                pickle.dump((input_features, is_rel), f)
        
        for key in test_data_raw:
            scores = np.arange([test_data_raw[key][i]['score'] for i in range(300)])
            stats = np.array(stats_drmm[key])
            input_features = np.column_stack((scores, stats))
            is_rel = [1 if test_data_raw[key][i]['doc_id'] in gt[key] else 0
                        for i in range(300)]
            X_test.append(input_features.tolist())
            y_test.append(is_rel)
            if not os.path.exits('{}/bicut_drmm_s{}_test/'.format(STATS_BASE, split)): 
                os.mkdir('{}/bicut_drmm_s{}_test/'.format(STATS_BASE, split))
            with open('{}/bicut_drmm_s{}_test/qid_{}.pkl'.format(STATS_BASE, split, key), 'wb') as f:
                pickle.dump((input_features, is_rel), f)